<a href="https://colab.research.google.com/github/konderal333/HGT-2022-EmDomArDon/blob/main/Scraping_and_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/konderal333/HGT-2022-EmDomArDon.git

Cloning into 'HGT-2022-EmDomArDon'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 201 (delta 24), reused 17 (delta 5), pack-reused 151
Receiving objects: 100% (201/201), 21.68 MiB | 13.92 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Checking out files: 100% (14/14), done.


In [ ]:
#Google drive for bigger data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Scraping

In [ ]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [ ]:
# This times ten is the number of articles to scrape 
cikkszam = 1000

# 2000 articles need approx 20 minutes to scrape

In [ ]:
page = requests.get("https://telex.hu/legfrissebb")   
soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify

titles = []
headline = []
textbody = []



#for the first page
links = soup.find_all('a',{'class':'list__item__title'})
heads = soup.find_all('p',{'class':'list__item__lead hasHighlight'})
for index, link in enumerate(links):
    
    #headline
    head = heads[index].text
    headline.append(head)
    
    #htmlbody of teh article
    link = link.attrs['href'] 
    newpage = requests.get(f"https://telex.hu{link}")
    newpage_html = BeautifulSoup(newpage.text, 'html.parser')
    newpage_html.prettify
    
    #title
    title = newpage_html.find('h1')
    titles.append(title.text)
    
    #textbody
    article = newpage_html.find('div', {'class':'article-html-content'}) 
    article_text = article.get_text()
    textbody.append(article_text)

    
# For the other pages
for pagenumber in range(2,cikkszam+1):
    page = requests.get(f"https://telex.hu/legfrissebb/?oldal={pagenumber}")   
    soup = BeautifulSoup(page.text, 'html.parser')
    soup.prettify
    
    links = soup.find_all('a',{'class':'list__item__title'})
    heads = soup.find_all('p',{'class':'list__item__lead hasHighlight'})
    for index, link in enumerate(links):

        #headline
        head = heads[index].text
        headline.append(head)

        #htmlbody of teh article
        link = link.attrs['href'] 
        newpage = requests.get(f"https://telex.hu{link}")
        newpage_html = BeautifulSoup(newpage.text, 'html.parser')
        newpage_html.prettify

        #title
        title = newpage_html.find('h1').text
        titles.append(title)

        #textbody
        article = newpage_html.find('div', {'class':'article-html-content'}) 
        article_text = article.get_text()
        textbody.append(article_text)

df_orig = pd.DataFrame({'Title':titles, 'Headline':headline, 'Textbody':textbody})       

#We don't need the Ukranian - Russian livefeed
df_orig.drop([0], inplace=True)
print('kész')

kész


In [ ]:
df_orig.to_csv('cikkek_10k.csv', index=False)

# Data Cleaning

In [ ]:
try:
  df_orig = pd.read_csv("/content/HGT-2022-EmDomArDon/cikkek_10k.csv")
except:
  raise ValueError("Upload it here or to Google Drive.")

In [ ]:
# replacing \n and \t characters

df = df_orig.astype(str)
df = df.apply(lambda x: x.replace('\n', " ", regex=True).replace('\t', " ", regex=True))

# dropping articles based on their length in tokens (words)

bad_length_indexes = []

for index, row in df.iterrows():
  if len(row['Textbody'].split(" ")) < 10:
    bad_length_indexes.append(index)


df = df.drop(bad_length_indexes)
df = df.reset_index(drop=True)
df

,Title,Headline,Textbody
0,Fájóan aktuális és elviselhetetlenül fe...,A friss Nobel-díjas Annie Ernaux könyvéből kés...,Az Esemény hazai szeptember végi mozis premier...
1,Brutális mértékben drágul a liba- és ka...,"Majdnem duplájába kerül, mint tavaly. Drágább ...",Majdnem duplájába kerülhet a Márton-napi libas...
2,"Két góllal vezetett a Bayern München, a...",Neuer kissé bizonytalanul jött ki az utolsó pe...,"Nagyon úgy tűnt, hogy a Bayern München ismét l..."
3,Antiszemita tartalom miatt korlátozták ...,"A rapper P. Diddyről állította azt, hogy a hát...",Korlátozást rakott a Meta Kanye West Instagram...
4,Meghalt egy néző az ózdi ralin,Elsodorta az egyik autó.,"Elsodortak egy nézőt a hétvégi ózdi ralin, és ..."
...,...,...,...
9828,Nyomozás indult a külügyminisztériumot ...,A Nemzeti Nyomozó Iroda a Direkt36 tényfeltáró...,A Nemzeti Nyomozó Iroda a Direkt36 tényfeltáró...
9829,Örökre lezárják az autósok előtt a pest...,Karácsony Gergely főpolgármester a választások...,„A pesti alsó rakpart tervezése során volt az ...
9830,"Jogerős: Nem bizonyítható, hogy a Megol...","Így a párt nem sértette meg a törvényt, hiszen...",A Kúria megváltoztatta a Nemzeti Választási Bi...
9831,"Két hétre leállt az oltási akció, mosta...",A kórházi oltópontok csak keddenként és péntek...,"Április elején írtuk meg, hogy a friss napi ad..."


In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.5 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3156027 sha256=f32c0414c23d059ed5dfdf5e224f21eb343bc4e3cc1818a211850187564a86a8
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
# Language detection using FastText: if not Hungarian, we drop it

import fasttext as ft

# fasttext model
# download and upload this manually or use Google Drive since github can only store up to 100 MB:
# https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

try:
  ft_model = ft.load_model("/content/drive/MyDrive/HGT/lid.176.bin")
except:
  raise ValueError("Upload it to Google Drive and mount it or upload it here!")


not_hun_indexes = []
check_manually_indexes = []

for index, row in df.iterrows():
  try:
    lang = ft_model.predict([row['Textbody']])
    if lang[0][0][0] != '__label__hu':
      not_hun_indexes.append(index)
  except:
    # manual checking
    check_manually_indexes.append(index)
    
df = df.drop(not_hun_indexes)

df = df.reset_index(drop=True)
df   



,Title,Headline,Textbody
0,Fájóan aktuális és elviselhetetlenül fe...,A friss Nobel-díjas Annie Ernaux könyvéből kés...,Az Esemény hazai szeptember végi mozis premier...
1,Brutális mértékben drágul a liba- és ka...,"Majdnem duplájába kerül, mint tavaly. Drágább ...",Majdnem duplájába kerülhet a Márton-napi libas...
2,"Két góllal vezetett a Bayern München, a...",Neuer kissé bizonytalanul jött ki az utolsó pe...,"Nagyon úgy tűnt, hogy a Bayern München ismét l..."
3,Antiszemita tartalom miatt korlátozták ...,"A rapper P. Diddyről állította azt, hogy a hát...",Korlátozást rakott a Meta Kanye West Instagram...
4,Meghalt egy néző az ózdi ralin,Elsodorta az egyik autó.,"Elsodortak egy nézőt a hétvégi ózdi ralin, és ..."
...,...,...,...
9606,Nyomozás indult a külügyminisztériumot ...,A Nemzeti Nyomozó Iroda a Direkt36 tényfeltáró...,A Nemzeti Nyomozó Iroda a Direkt36 tényfeltáró...
9607,Örökre lezárják az autósok előtt a pest...,Karácsony Gergely főpolgármester a választások...,„A pesti alsó rakpart tervezése során volt az ...
9608,"Jogerős: Nem bizonyítható, hogy a Megol...","Így a párt nem sértette meg a törvényt, hiszen...",A Kúria megváltoztatta a Nemzeti Választási Bi...
9609,"Két hétre leállt az oltási akció, mosta...",A kórházi oltópontok csak keddenként és péntek...,"Április elején írtuk meg, hogy a friss napi ad..."


In [ ]:
check_manually_indexes

[]

In [ ]:
df.to_csv("cikkek_10k_cleanedv1.csv", index=False)

# Modeling